In [1]:
%pylab inline
import os
import sys
import numpy as np
import importlib

%cd ..
p = os.getcwd()
print("path:" + p)
if p not in sys.path:
    sys.path.append(p)
    
from cnn_sys_ident.data import Dataset, MonkeyDataset
from cnn_sys_ident.vggsysid import VggTransfer

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
c:\Cadena2019PlosCB
path:c:\Cadena2019PlosCB


c:\Cadena2019PlosCB\.research6\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Load Data

In [2]:
# data_dict = Dataset.load_data()
# data_dict = Dataset.manage_repeats(data_dict)
# data_dict = Dataset.preprocess_nans(data_dict)
# data_dict = Dataset.add_train_test_types(data_dict, types_train='all', types_test='all')

# With a wrapper function
data_dict = Dataset.get_clean_data()

In [3]:
data = MonkeyDataset(data_dict, seed=1000, train_frac=0.8 ,subsample=2, crop = 30)

### Define the Model

In [4]:
model = VggTransfer(data, log_dir='monkey', log_hash='vgg', obs_noise_model='poisson')

In [5]:
print('Log dir: %s' % model.log_hash)
_, test_responses = data.test_av()
_, val_responses, real_val_resps = data.val()
_, tr_responses, real_tr_resps = data.train()

val_array = data.nanarray(real_val_resps,val_responses)
tr_array = data.nanarray(real_tr_resps,tr_responses)
print('Average variances | validation set: %f | test set: %f' % (np.nanmean(np.nanvar(val_array, axis=0)), np.nanmean(np.nanvar(test_responses, axis=0))))

Log dir: vgg


c:\Cadena2019PlosCB\cnn_sys_ident\data.py:180: RuntimeWarning: Mean of empty slice
  return self.images_test, np.nanmean(self.nanarray(self.real_resps_test,self.responses_test),axis=0)


Average variances | validation set: 2.586168 | test set: 1.457066


### Build the Model

In [6]:
model.build(name_readout_layer='conv3/conv3_1', 
          smooth_reg_weight=0.1, 
          sparse_reg_weight=0.01, 
          group_sparsity_weight=0.01,
          output_nonlin_smooth_weight=-1,
          b_norm=True)

c:\Cadena2019PlosCB\.research6\Lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
c:\Cadena2019PlosCB\cnn_sys_ident\vggsysid.py:64: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  vgg_feats_bn = tf.compat.v1.layers.batch_normalization(vgg_features, training = self.is_training,\


Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Restoring parameters from c:\Cadena2019PlosCB\vgg_weights/vgg_normalized.ckpt


### Train the Model

In [7]:
# Training the network
learning_rate=1e-4
for lr_decay in range(3):
    training = model.train(max_iter=10000,
                         val_steps=100, 
                         save_steps=10000,
                         early_stopping_steps=10,
                         batch_size=256,
                         learning_rate=learning_rate)
    for (i, (logl, total_loss, mse, pred)) in training:
        print('Step %d | Total loss: %s | %s: %s | MSE: %s | Var(y): %s' % (i, total_loss, model.obs_noise_model, logl, mse, np.mean(np.var(pred, axis=0))))
    learning_rate /= 3  # Learning rate decays to one third once it stops improving
    print('Reducing learning rate to %f' % learning_rate)
print('Done fitting')

KeyboardInterrupt: 

### Test Performance of the Model

In [ ]:
model.performance_test()
eve = model.eve.mean()
print('Explainable variance explained on test set: {}'.format(eve))

In [ ]:
model.performance_val()
eve_val = model.eve_val.mean()
print('Explainable variance explained on validation set: {}'.format(eve_val))

In [ ]:
avg_correlation_valset = model.evaluate_avg_corr_val()
print('Mean single trial correlation on validation set: {}'.format(avg_correlation_valset))